In [ ]:
%pip install langchain-ollama

In [1]:
import pandas as pd

df = pd.read_csv('../dataset/src.csv')

In [5]:
df_trimmed = df[['recipe_id', 'name']]

In [2]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.1",
    temperature=0.2
)

In [3]:
system_prompt = '''
You are an expert chef who writes structured descriptions of dishes for a recipe book. For every dish, you must include the following details:

Dish Name: Provide the exact name of the dish as provided in the input.

Short Description: Write a brief description of the dish in 1-3 sentences. Start the description with the dish name in square brackets, followed by the description. Mention key ingredients, the style of preparation, and any special characteristics of the dish (e.g., flavor profile, texture, or cuisine type).

Format: [###Dish Name###] ###Short Description###
Ensure every response follows this format.
'''

system_message = [("system", system_prompt)]

In [ ]:
# example prompt
user_prompt = "Cheeese Pizza"
user_message = [("human", user_prompt)]
messages = system_message + user_message
description = llm.invoke(messages)
description

In [ ]:
df_trimmed['generated_description'] = None

In [ ]:
sum_input_tokens = 0
sum_output_tokens = 0

def generate(dish_name):
    global sum_input_tokens, sum_output_tokens
    response = llm.invoke(system_message + [('human', dish_name)])
    sum_input_tokens += response.usage_metadata['input_tokens']
    sum_output_tokens += response.usage_metadata['output_tokens']
    return response.content

for i in range(len(df)):
    dish_name = df_trimmed['name'][i]
    df_trimmed.loc[i, 'generated_description'] = generate(dish_name)

# usage stats
print("Input Tokens {}\nOutput Tokens: {}\nTotal Tokens: {}".format(
    sum_input_tokens,
    sum_output_tokens,
    sum_input_tokens + sum_output_tokens
))

In [ ]:
df_trimmed.tail()

In [14]:
df_trimmed.to_csv('src_with_desc.csv')